<a href="https://colab.research.google.com/github/spegas/Dacon/blob/main/Get_Price_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install finance-datareader

In [2]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9672 sha256=0aac2173c28b79e092605c1f3de19f892fa6b78d1fd1a7c2d53fbe7a04fe7ab3
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [9]:
import pandas as pd
import os
import FinanceDataReader as fdr
import wget

from zipfile import ZipFile
from os.path import basename

In [7]:
if not os.path.isdir("base"):
  print("base 디렉토리 생성")
  os.makedirs("base")

if not os.path.isdir("price"):
  print("price 디렉토리 생성")
  os.makedirs("price")

In [10]:
stock_list_file_url = "https://raw.githubusercontent.com/spegas/Dacon/main/stock_list.csv"
if os.path.exists('base/stock_list.csv'):
  os.remove('base/stock_list.csv')
  print('이전에 저장된 주식 종목 리스트 파일을 삭제 합니다.')
wget.download(stock_list_file_url, 'base/stock_list.csv')
print('주식 종목 리스트 파일 다운로드 완료')

주식 종목 리스트 파일 다운로드 완료


In [ ]:
stock_list_name = 'base/stock_list.csv'
pd_stock_list = pd.read_csv(stock_list_name)
pd_stock_list['종목코드'] = pd_stock_list['종목코드'].apply(lambda x : str(x).zfill(6))
pd_stock_list.head(10)

In [14]:
start_date = '20200101'
end_date = '20211031'

In [ ]:
for f_index in range(0, pd_stock_list.shape[0]):
  f_code = pd_stock_list.iloc[f_index]['종목코드']
  f_name = pd_stock_list.iloc[f_index]['종목명']
  price_filename = str('price/') + str(f_code) + str('_price.csv')
  if not os.path.exists(price_filename):
    f_price = fdr.DataReader(f_code, start = start_date, end = end_date)
    f_price.to_csv(price_filename)
    print('[종목코드 : %s, 종목명 : %s] 가격 정보 저장 완료' % (str(f_code), str(f_name)))
  else:
    print('[종목코드 : %s, 종목명 : %s] 가격 정보 파일 존재' % (str(f_code), str(f_name)))

In [25]:
# Zip the files from given directory that matches the filter
def zipFilesInDir(dirName, zipFileName, filter):
   # create a ZipFile object
   with ZipFile(zipFileName, 'w') as zipObj:
       # Iterate over all the files in directory
       for folderName, subfolders, filenames in os.walk(dirName):
           for filename in filenames:
               if filter(filename):
                   # create complete filepath of file in directory
                   filePath = os.path.join(folderName, filename)
                   # Add file to zip
                   zipObj.write(filePath, basename(filePath))

In [29]:
zipFilesInDir('price', 'price.zip', lambda name : 'csv' in name)